In [ ]:
import os 
import sys 

cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

from functools import cache 
import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

from utils_notebook.utils import ddf, load_subgraph, remove_prefix
from utils_notebook.vega import output_chart
sg: Subgrounds
bs: Subgraph
sg, bs = load_subgraph()
def query_farmers_market_snaps(refresh=None):    
    snaps = bs.Query.podMarketplaceDailySnapshots(first=10000, orderBy="season", orderDirection='asc')
    bs.PodMarketplaceDailySnapshot.total_bean_vol = bs.PodMarketplaceDailySnapshot.totalBeanVolume / 1e6
    bs.PodMarketplaceDailySnapshot.total_pod_vol = bs.PodMarketplaceDailySnapshot.totalPodVolume / 1e6
    bs.PodMarketplaceDailySnapshot.total_pod_listing_vol = bs.PodMarketplaceDailySnapshot.totalPodsFilled / 1e6
    bs.PodMarketplaceDailySnapshot.total_pod_order_vol = bs.PodMarketplaceDailySnapshot.totalOrdersFilled / 1e6
    df = sg.query_df([
        snaps.timestamp, 
        snaps.season, 
        snaps.total_bean_vol, # total bean volume for filled orders / listings 
        snaps.total_pod_vol, # total pod volume for filled orders / listings 
        snaps.total_pod_listing_vol, # total pod volume for filled listings 
        snaps.total_pod_order_vol, # total pod volume for filled orders
    ], pagination_strategy=ShallowStrategy)
    df = remove_prefix(df, 'podMarketplaceDailySnapshots_')
    return df 
df_snaps = query_farmers_market_snaps()
df_snaps['total bean vol'] = df_snaps['total_bean_vol']
df_snaps['total pod listing vol'] = df_snaps['total_pod_listing_vol']
df_snaps['total pod order vol'] = df_snaps['total_pod_order_vol']
df_snaps['total pod vol'] = df_snaps['total_pod_vol']
value_vars = [
    'total_bean_vol', 'total_pod_vol', 'total_pod_listing_vol', 'total_pod_order_vol'
]
label_vars = [
    'total bean vol', 'total pod listing vol', 'total pod order vol', 'total pod vol'
] 
id_vars = ['timestamp', 'season'] + label_vars
df_snaps = df_snaps.melt(id_vars=id_vars, value_vars=value_vars).sort_values("season")
rm_indices = []
for v in value_vars: 
    rm_indices = rm_indices + list(
        np.argwhere(
            ((df_snaps.season == 6074) & (df_snaps.variable == v)).values
        ).ravel()[:-1]
    )
df_snaps = df_snaps.iloc[[i for i in range(len(df_snaps)) if i not in rm_indices]]
df_snaps = df_snaps.loc[df_snaps.season >= 4357]

area = alt.Chart(df_snaps).transform_filter(
    datum.variable != 'total_pod_vol'
).transform_stack(
    stack="value", 
    as_=['value_1', 'value_2'], 
    groupby=['season'],
    sort=[alt.SortField('variable', 'descending')]
).mark_area(point='transparent').encode(
    x="season:O", 
    y="value_1:Q", 
    y2="value_2:Q", 
    color="variable:N", 
    tooltip=[alt.Tooltip(f'{e}:Q', format=",d") for e in label_vars]
).properties(width=700)

line = alt.Chart(df_snaps).transform_filter(
    datum.variable == 'total_pod_vol'
).mark_line().encode(
    x="season:O", 
    y="value:Q", 
    color="variable:N", 
).properties(width=700)

c = area + line
c
output_chart(c)
